In [1]:
!nvidia-smi

Tue Jan  9 20:50:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.36                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    On  | 00000000:01:00.0  On |                  N/A |
| N/A   50C    P8              17W /  80W |   3063MiB /  8192MiB |     30%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -qq diffusers transformers xformers

In [ ]:
!pip install -qq opencv-contrib-python
!pip install -qq controlnet_aux

# Load QR Code ControlNet

In [ ]:
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline

# Load the QR code to generate the corresponding ControlNet model
controlnet = ControlNetModel.from_pretrained("Nacholmo/controlnet-qr-pattern-v2")
pipeline = StableDiffusionControlNetPipeline.from_pretrained(
	"runwayml/stable-diffusion-v1-5", controlnet=controlnet
)

In [ ]:
import torch
from PIL import Image
from diffusers import DDIMScheduler
from diffusers.utils import load_image

pipeline.enable_xformers_memory_efficient_attention()
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.enable_model_cpu_offload()

# Scale the image
def resize_for_condition_image(input_image: Image, resolution: int):
    input_image = input_image.convert("RGB")
    W, H = input_image.size
    k = float(resolution) / min(H, W)
    H *= k
    W *= k
    H = int(round(H / 64.0)) * 64
    W = int(round(W / 64.0)) * 64
    img = input_image.resize((W, H), resample=Image.LANCZOS)
    return img

In [ ]:
# Process the original QR code image, you can use your QR code link, or upload the local QR code image to Colab
init_image = load_image("https://media.discordapp.net/attachments/1092064974794391644/1194270793937457222/ninja_geek_qr_code_style_one.png?ex=65afbe59&is=659d4959&hm=2379cc23f4aabe15e2555c87aa43956b908a2723af60ba76d86999039e2b46d0&=&format=webp&quality=lossless&width=1005&height=1005")
init_image = resize_for_condition_image(init_image, 768)
generator = torch.manual_seed(1025)
init_image

In [ ]:
# You can replace it with your prompt
prompt = ["robot costume, best quality, extremely detailed", "masterpiece, forest, jungle, trees, mist, fog, water, river, vines, photorealistic"]

generator = torch.Generator("cuda").manual_seed(1025)

# Generate QR code
output = pipeline(
    prompt,
    init_image,
    negative_prompt=["ugly, disfigured, low quality, blurry, nsfw"] * len(prompt),
    generator=generator,
    num_inference_steps=20,
)


In [ ]:
# Used to display image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

image_grid(output.images, 1, 2)

# Let's test other prompts

In [ ]:
# You can replace it with your prompt
prompt = ["sea wave, best quality, extremely detailed", "masterpiece, best quality, everyday life in the cloud kingdom"]

generator = torch.Generator("cuda").manual_seed(1025)

# Generate QR code
output = pipeline(
    prompt,
    init_image,
    negative_prompt=["ugly, disfigured, low quality, blurry, nsfw"] * len(prompt),
    generator=generator,
    num_inference_steps=20,
    guidance_scale = 5.0,
)

In [ ]:
# Used to display image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

image_grid(output.images, 1, 2)

In [ ]:
pipeline